<h1>光頭傑夫製作</h1><img src="https://bl3302files.storage.live.com/y4mccLq6gAm5ULg2cdCQ-Bl3y0Lw7mmtkvcdqInwwg3MAPeDpabT_YVDs7vVN8tvdaw4meL8i33B42YbUECTNMGkmykUQOC9uJxAC6jE_XkaP7K4BTP1xFvyONy_h8LOGV_QW_2CqIwThrYu1JxnujMB7IUR3y7NLJXaXRZMQekQL80-WZuuGw7EIsdneNGUnE0?width=100&height=91&cropmode=none" style='width:100px;margin:0'/>

### <span style="color:red ">設定目標股票
----
### [查詢股票代碼](https://tw.stock.yahoo.com/h/kimosel.php)

In [1]:
#########################################
#請填下面資料
from datetime import date,timedelta
目標股票代碼='2330.tw'
分析開始日期='2021-1-1'
分析結束日期=str(date.today() + timedelta(1))

#設立投資本金
#你可以根據你想要模擬的投資標的股價作適當調整
cash=10000000

##########################################

### <span style="color:red ">Colab安裝套件

In [2]:
! pip install matplotlib==3.2.2
! pip install backtrader
! pip install yfinance

In [ ]:
import matplotlib
matplotlib.__version__

### <span style="color:red ">以下是我們後續輸出分析結果需要的方程式:
---    
    1. 每次你要輸出策略的績效結果時，下面這格你都要跑

In [3]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_closed = ''
    total_won =''
    total_lost =''
    win_streak = ''
    lose_streak = ''
    pnl_net =''
    strike_rate =''
    
    if len(analyzer.total)>2:
        total_closed = analyzer.total.closed
        total_won = analyzer.won.total
        total_lost = analyzer.lost.total
        win_streak = analyzer.streak.won.longest
        lose_streak = analyzer.streak.lost.longest
        pnl_net = round(analyzer.pnl.net.total,2)
    
    if total_closed:
        strike_rate=round((total_won / total_closed) * 100,2)
    else:
        strike_rate='沒有完整交易'
    print("策略運作分析結果:")
    print(f'獲利累積次數 : {str(total_won)}')
    print(f'損失累積次數 : {str(total_lost)}')
    print(f'獲利損失次數比 : {str(strike_rate)}')
    print(f'最高連續獲利交易次數 : {str(win_streak)}')
    print(f'最高連續損失交易次數 : {str(lose_streak)}')
    print(f'淨利結算 : {str(pnl_net)}')
    

## <span style="color:red ">  <策略1-with Commision>用RSI，來做個簡單的回測策略吧!
---
### 先回想一下，你還記的20-day RSI是什麼意思嗎?
    
    我們測定策略如下:
    1.如果今天的20-day RSI<20，那我們就買進一股的股票。
    2.如果今天的20-day RSI>80，那我們就賣掉一股的股票。
    3.任何時候，我們手上最多就一股股票。
    
    

### <span style="color:red "><策略1.1- with Commision>導入需求模組

In [4]:
from datetime import datetime
import pandas as pd
import yfinance as yf
import backtrader as bt

### <span style="color:red "><策略1.2- with Commision>建立交易策略

In [5]:
class RsiStrategy(bt.Strategy):
    params = dict(
        ma_period=20,
        rsi_uplimit=80,
        rsi_downlimit=20,
    )

    def __init__(self):
        # keep track of close price in the series
        self.data_close = self.datas[0].close

        # keep track of pending orders/buy price/buy commission
        self.order = None
        self.price = None
        self.comm = None

        # add a simple rsi indicator
        self.rsi = bt.indicators.RSI_Safe(self.data.close, period=self.params.ma_period)

    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'買單執行 --- 買價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')
                self.price = order.executed.price
                self.comm = order.executed.comm
            else:
                self.log(f'賣單執行 --- 賣價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')

        # report failed order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失敗')

        # set no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'運作結果 --- 獲利狀況: {trade.pnl:.2f}, 淨獲利: {trade.pnlcomm:.2f}')

    def next(self):
        # do nothing if an order is pending
        if self.order:
            return

        # check if there is already a position
        if not self.position:
            # buy condition
            if self.rsi < self.params.rsi_downlimit:
                self.log(f'開出買單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.buy(size=1000)
        else:
            # sell condition
            if self.rsi > self.params.rsi_uplimit:
                self.log(f'開出賣單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.sell(size=1000)

### <span style="color:red "><策略1.3- with Commision>抓取目標股票的歷史資料

In [6]:
data= yf.download(  目標股票代碼,              #股票代碼
                    start=分析開始日期,      #資料開始日期
                    end=分析結束日期,         #資料結束日期
                    progress=False
                 )
data=bt.feeds.PandasData(dataname=pd.DataFrame({'open': data['Open'], 
                   'high': data['High'],
                   'low': data['Low'],
                   'close': data['Close'],
                   'volume': data['Volume'],
                  }))


### <span style="color:red "><策略1.4- with Commision>建立台股交易費用計算模式

In [7]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)

### <span style="color:red "><策略1.5- with Commision>建立回測分析「大腦」

In [8]:
# create a Cerebro entity
cerebro = bt.Cerebro()

# set up the backtest
cerebro.adddata(data)
cerebro.broker.setcash(cash)
cerebro.addstrategy(RsiStrategy)

#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)

cerebro.broker.addcommissioninfo(comminfo)


### <span style="color:red "><策略1.6- with Commision>讓Analyzers來幫我診斷策略細節...-開始回測

In [9]:
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

strategies = cerebro.run()
firstStrat = strategies[0]

# print the analyzers
printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
print(f'持有資產總值: {cerebro.broker.getvalue():.2f}')

策略運作分析結果:
獲利累積次數 : 
損失累積次數 : 
獲利損失次數比 : 沒有完整交易
最高連續獲利交易次數 : 
最高連續損失交易次數 : 
淨利結算 : 
持有資產總值: 1000000.00


### <span style="color:red "><策略1.7- with Commision>畫張圖勝過千言萬語!-圖形化表達

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.dpi']=80
cerebro.plot(iplot=False)

[[<Figure size 1368x720 with 5 Axes>]]

## <span style="color:red "><策略2- with Commision>我們不得不懷疑20-day RSI是一個好選擇嗎?
--------
    

### <span style="color:red "><策略2.2- with Commision>建立交易策略

In [11]:
class RsiStrategy(bt.Strategy):
    params = dict(
        ma_period=20, 
        rsi_uplimit=80,
        rsi_downlimit=20,
        totalValue=''
    )

    def __init__(self):
        # keep track of close price in the series
        self.data_close = self.datas[0].close

        # keep track of pending orders
        self.order = None
        self.price = None
        self.comm = None

        # add a simple moving average indicator
        self.rsi = bt.indicators.RSI_Safe(self.data.close, period=self.params.ma_period)

    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        # set no pending order
        self.order = None

    def next(self):
        # do nothing if an order is pending
        if self.order:
            return
                
        # check if there is already a position
        if not self.position:
            # buy condition
            if self.rsi < self.params.rsi_downlimit:
                self.order = self.buy(size=1000)
        else:
            # sell condition
            if self.rsi > self.params.rsi_uplimit:
                self.order = self.sell(size=1000)

    def stop(self):
        self.log(f'(RSI週期設定 = {self.params.ma_period:2d} RSI上限設定 = {self.params.rsi_uplimit:2d} RSI下限設定 = {self.params.rsi_downlimit:2d}) --- 淨利數值: {self.broker.getvalue()-cash:.2f}')
        self.params.totalValue=self.broker.getvalue()



### <span style="color:red "><策略2.4- with Commision>建立台股交易費用計算模式

In [12]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)

### <span style="color:red "><策略2.5- with Commision >建立回測分析「大腦」,讓Analyzers來幫我診斷策略細節...-開始回測

In [13]:
# create a Cerebro entity
cerebro = bt.Cerebro()
cerebro.optstrategy(RsiStrategy, ma_period=range(10, 60),rsi_uplimit=[65,70,75,80],rsi_downlimit=[5,10,15,20,25])
cerebro.adddata(data)
cerebro.broker.setcash(cash)



#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)


cerebro.broker.addcommissioninfo(comminfo)

cerebro.addanalyzer(bt.analyzers.TradeAnalyzer,_name="ta")

opt_runs = cerebro.run(maxcpus=1)


for run in opt_runs:
    for strategy in run:
        print(f'RSI週期設定 = {strategy.params.ma_period:2d} RSI上限設定 = {strategy.params.rsi_uplimit:2d} RSI下限設定 = {strategy.params.rsi_downlimit:2d}')
        try:
            printTradeAnalysis(strategy.analyzers.ta.get_analysis())
        except KeyError :
            print('沒有交易發生')
        print(f'持有資產總值: {strategy.params.totalValue:.2f}')
        print('--------------------------------------------------------')

2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 65 RSI下限設定 =  5) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 65 RSI下限設定 = 10) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 65 RSI下限設定 = 15) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 65 RSI下限設定 = 20) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 65 RSI下限設定 = 25) --- 淨利數值: 49346.65
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 70 RSI下限設定 =  5) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 70 RSI下限設定 = 10) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 70 RSI下限設定 = 15) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 70 RSI下限設定 = 20) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 70 RSI下限設定 = 25) --- 淨利數值: 27603.85
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 75 RSI下限設定 =  5) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 75 RSI下限設定 = 10) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 75 RSI下限設定 = 15) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 75 RSI下限設定 = 20) --- 淨利數值: 0.00
2021-07-09, (RSI週期設定 = 10 RSI上限設定 = 75 R

KeyboardInterrupt: 

## <span style="color:red "><策略3- with Commision>技術分析不是一直說什麼黃金交叉...什麼死亡交叉...
-----
    1.黃金交叉：短天期均線向上與長天期交叉，稱為黃金交叉。
    2.死亡交叉：短天期均線向下與長天期交叉，稱為死亡交叉。

### <span style="color:red "><策略3.2- with Commision>建立交易策略

In [ ]:
class RsiCross(bt.Strategy):
    params = dict(
        pfast=5,   # 短期均線週期
        pslow=10   # 長期均線週期
    )

    def __init__(self):
        self.data_close = self.datas[0].close
        
        self.order = None
        self.price = None
        self.comm = None

        
        rsi1 = bt.indicators.RSI_Safe(self.datas[0],period=self.p.pfast)  # 短期均線
        rsi2 = bt.indicators.RSI_Safe(self.datas[0],period=self.p.pslow)  # 長期均線
        self.crossover = bt.ind.CrossOver(rsi1, rsi2)

    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'買單執行 --- 買價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')
                self.price = order.executed.price
                self.comm = order.executed.comm
            else:
                self.log(f'賣單執行 --- 賣價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')

        # report failed order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失敗')

        # set no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'運作結果 --- 獲利狀況: {trade.pnl:.2f}, 淨獲利: {trade.pnlcomm:.2f}')

    def next(self):
        # do nothing if an order is pending
        if self.order:
            return

        # check if there is already a position
        if not self.position:
            # buy condition
            if self.crossover > 0:
                self.log(f'開出買單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.buy(size=1000)
        else:
            # sell condition
            if self.crossover < 0:
                self.log(f'開出賣單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.sell(size=1000)
    def stop(self):
        self.log(f'(RSI週期設定 = 短週期:{self.p.pfast:2d},長週期:{self.p.pslow:2d}) --- 最終資產價值: {self.broker.getvalue():.2f}')

### <span style="color:red "><策略3.4- with Commision>建立台股交易費用計算模式

In [ ]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)

### <span style="color:red "><策略3.5- with Commision>建立回測分析「大腦」

In [ ]:
# create a Cerebro entity
cerebro = bt.Cerebro()
cerebro.addstrategy(RsiCross)
cerebro.adddata(data)
cerebro.broker.setcash(cash)



#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)

cerebro.broker.addcommissioninfo(comminfo)



### <span style="color:red "><策略3.6- with Commision>讓Analyzers來幫我診斷策略細節...-開始回測

In [ ]:
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

strategies = cerebro.run()
firstStrat = strategies[0]

# print the analyzers
printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
print(f'持有資產總值: {firstStrat.broker.getvalue():.2f}')


### <span style="color:red "><策略3.7- with Commision>畫張圖勝過千言萬語!-圖形化表達

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = [19, 10]
cerebro.plot(iplot=False)

## <span style="color:red "><策略4- with Commision>上面是5日線和10日線的交叉結果，那如果我想要知道哪兩條線的配對交叉結果最好...
-----
###     我就是想知道:
        到底是RSI5穿過RSI20比較厲害?
        還是RSI20穿出RSI60比較厲害?

### <span style="color:red "><策略4.2- with Commision>建立交易策略

In [ ]:
class RsiCross(bt.Strategy):
    params = dict(
        pfast=5,   # 短期均線週期
        pslow=10,   # 長期均線週期
        totalValue=''
    )

    def __init__(self):
        self.data_close = self.datas[0].close
        
        self.order = None
        self.price = None
        self.comm = None

        
        rsi1 = bt.indicators.RSI_Safe(self.datas[0],period=self.p.pfast)  # 短期均線
        rsi2 = bt.indicators.RSI_Safe(self.datas[0],period=self.p.pslow)  # 長期均線
        self.crossover = bt.ind.CrossOver(rsi1, rsi2)

    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.price = order.executed.price
                self.comm = order.executed.comm

        # set no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return


    def next(self):
        # do nothing if an order is pending
        if self.order:
            return

        # check if there is already a position
        if not self.position:
            # buy condition
            if self.crossover > 0:
                self.order = self.buy(size=1000)
        else:
            # sell condition
            if self.crossover < 0:
                self.order = self.sell(size=1000)
    def stop(self):
        self.log(f'(RSI週期設定 = 短週期:{self.p.pfast:2d},長週期:{self.p.pslow:2d}) --- 淨利數值: {self.broker.getvalue()-cash:.2f}')
        self.params.totalValue=self.broker.getvalue()

### <span style="color:red "><策略4.4- with Commision>建立台股交易費用計算模式

In [ ]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)

### <span style="color:red "><策略4.5- with Commision>建立回測分析「大腦」,讓Analyzers來幫我診斷策略細節...-開始回測

In [ ]:
cerebro = bt.Cerebro()
cerebro.optstrategy(RsiCross, pfast=[5,7, 10,12, 15],pslow=[20,25,30,35,40,45])
cerebro.adddata(data)
cerebro.broker.setcash(cash)


#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)


cerebro.broker.addcommissioninfo(comminfo)

cerebro.addanalyzer(bt.analyzers.TradeAnalyzer,_name="ta")

opt_runs = cerebro.run(maxcpus=1)


for run in opt_runs:
    for strategy in run:
        print(f'RSI短週期設定:{strategy.params.pfast} RSI長週期設定:{strategy.params.pslow}')
        try:
            printTradeAnalysis(strategy.analyzers.ta.get_analysis())
        except KeyError :
            print('沒有交易發生')
        print(f'持有資產總值: {strategy.params.totalValue:.2f}')
        print('--------------------------------------------------------')